In [1]:
#!pip install datasets
#!pip install --upgrade --force-reinstall huggingface_hub

In [2]:
import torch
import torchvision
from torchvision.transforms import transforms, Lambda, Resize
from torchvision import datasets, transforms, models
import os
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Specify where to find the data preparation class
import sys
sys.path.append('Preparation')
from Preparation import CustomDataLoader

In [3]:
# ResNet50 training data (ImageNet) properties
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
#DIMENSIONS = 3
#SIZE = 256

In [4]:
dataloaders = {x: CustomDataLoader(data_path="FER2013", batch_size=32, dataset_type=x, mean=MEAN, std=STD, dimensions=3).data_loader for x in ['train', 'test']}
dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'test']}
print(dataset_sizes)

class_names = dataloaders['train'].dataset.classes
print(class_names)

# Confirm correct data load
print("Train Data Loader:")
for batch_idx, (inputs, labels) in enumerate(dataloaders['train']):
    print("Batch Index:", batch_idx)
    print("Inputs Shape:", inputs.shape)
    print("Labels Shape:", labels.shape)
    # Print the first few labels in the batch
    print("Labels:", labels[:5])
    # Break after printing a few batches
    if batch_idx == 2:
        break

{'train': 877, 'test': 225}
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Train Data Loader:
Batch Index: 0
Inputs Shape: torch.Size([32, 3, 299, 299])
Labels Shape: torch.Size([32])
Labels: tensor([3, 0, 2, 6, 5])
Batch Index: 1
Inputs Shape: torch.Size([32, 3, 299, 299])
Labels Shape: torch.Size([32])
Labels: tensor([0, 0, 2, 2, 6])
Batch Index: 2
Inputs Shape: torch.Size([32, 3, 299, 299])
Labels Shape: torch.Size([32])
Labels: tensor([6, 4, 5, 3, 3])


In [5]:
# Load the pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the final classification layer
for name, param in model.named_parameters():
    if "fc" in name:  # Unfreeze the final classification layer
        param.requires_grad = True
    else:
        param.requires_grad = False

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # Use all parameters


# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

C:\Users\suren\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\suren\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    for phase in ['train', 'test']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print("Training complete!")

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

train Loss: 59.7414 Acc: 9.4413


 10%|███████▌                                                                    | 1/10 [2:16:39<20:29:52, 8199.21s/it]

test Loss: 52.9880 Acc: 11.7600
train Loss: 51.6626 Acc: 11.8506


 20%|███████████████▏                                                            | 2/10 [4:26:04<17:39:12, 7944.09s/it]

...

test Loss: 47.0777 Acc: 13.6933
train Loss: 47.1678 Acc: 13.8734


 90%|████████████████████████████████████████████████████████████████████▍       | 9/10 [19:35:04<2:13:36, 8016.44s/it]

test Loss: 46.8641 Acc: 13.8533
train Loss: 46.7471 Acc: 14.0912


100%|█████████████████████████████████████████████████████████████████████████████| 10/10 [21:53:01<00:00, 7878.17s/it]

test Loss: 46.6677 Acc: 14.0133
Training complete!


In [7]:
# Save the model
torch.save(model.state_dict(), 'ResNet50.pth')